In [21]:
import uuid
import os
from dotenv import load_dotenv

from google.adk.agents import LlmAgent, ParallelAgent, SequentialAgent
from google.adk.tools import google_search
from google.adk.models.lite_llm import LiteLlm
from google.adk.runners import InMemoryRunner
from google.genai import types

load_dotenv()

# 使用自定义 OpenAI-compatible 模型
GEMINI_MODEL = LiteLlm(
    model="gemini-2.5-flash",
    api_base=os.getenv("OPENAI_API_BASE"),
    api_key=os.getenv("OPENAI_API_KEY"),
)

# 定义研究子 agent
researcher_agent_1 = LlmAgent(
    name="RenewableEnergyResearcher",
    model=GEMINI_MODEL,
    instruction="Research renewable energy...",
    tools=[google_search],
    output_key="renewable_energy_result",
)

researcher_agent_2 = LlmAgent(
    name="EVResearcher",
    model=GEMINI_MODEL,
    instruction="Research electric vehicles...",
    tools=[google_search],
    output_key="ev_technology_result",
)

researcher_agent_3 = LlmAgent(
    name="CarbonCaptureResearcher",
    model=GEMINI_MODEL,
    instruction="Research carbon capture...",
    tools=[google_search],
    output_key="carbon_capture_result",
)

# Parallel + Sequential workflow
parallel_research_agent = ParallelAgent(
    name="ParallelWebResearchAgent",
    sub_agents=[researcher_agent_1, researcher_agent_2, researcher_agent_3],
)

merger_agent = LlmAgent(
    name="SynthesisAgent",
    model=GEMINI_MODEL,
    instruction="""
Combine the following:
Renewable: {renewable_energy_result}
EV: {ev_technology_result}
Carbon: {carbon_capture_result}
Output a structured report only using these.
""",
)

sequential_pipeline_agent = SequentialAgent(
    name="ResearchAndSynthesisPipeline",
    sub_agents=[parallel_research_agent, merger_agent],
)

root_agent = sequential_pipeline_agent

# Notebook friendly: 单元格直接用顶层 await
user_id = "user_123"
session_id = str(uuid.uuid4())

runner = InMemoryRunner(root_agent, app_name="SustainableTechApp")

# ✨ This is the key fix — create the session before running
await runner.session_service.create_session(
    app_name="SustainableTechApp",
    user_id=user_id,
    session_id=session_id,
)

# 构造用户消息
user_msg = types.Content(
    role="user",
    parts=[types.Part(text="Start the research on sustainable technologies.")]
)

# 运行
async for event in runner.run_async(
    user_id=user_id,
    session_id=session_id,
    new_message=user_msg
):
    if event.is_final_response() and event.content:
        # 输出最终报告
        if event.content.text:
            print("\n📝 FINAL REPORT:\n", event.content.text)
        else:
            print("\n📝 FINAL REPORT:\n", "".join(p.text for p in event.content.parts))

20:57:00 - LiteLLM:ERROR: vertex_llm_base.py:550 - Failed to load vertex credentials. Check to see if credentials containing partial/invalid information. Error: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.
Traceback (most recent call last):
  File "/Users/haoquanzhai/miniforge3/envs/apple-ai/lib/python3.12/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", line 546, in get_access_token
    _credentials, credential_project_id = self.load_auth(
                                          ^^^^^^^^^^^^^^^
  File "/Users/haoquanzhai/miniforge3/envs/apple-ai/lib/python3.12/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", line 121, in load_auth
    creds, creds_project_id = self._credentials_from_default_auth(
                              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/haoquanzhai/miniforge3/envs/apple-ai/lib/python3.12/site-pack

  + Exception Group Traceback (most recent call last):
  |   File "/Users/haoquanzhai/miniforge3/envs/apple-ai/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3697, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/var/folders/x0/l2mn1t7s04v6q7q6yj5w_bf80000gn/T/ipykernel_22700/373213617.py", line 90, in <module>
  |     async for event in runner.run_async(
  |   File "/Users/haoquanzhai/miniforge3/envs/apple-ai/lib/python3.12/site-packages/google/adk/runners.py", line 519, in run_async
  |     async for event in agen:
  |   File "/Users/haoquanzhai/miniforge3/envs/apple-ai/lib/python3.12/site-packages/google/adk/runners.py", line 507, in _run_with_trace
  |     async for event in agen:
  |   File "/Users/haoquanzhai/miniforge3/envs/apple-ai/lib/python3.12/site-packages/google/adk/runners.py", line 739, in _exec_with_plugin
  |     async for event in agen:
  |   File "/Users/haoquanzhai/miniforge3/envs/apple-ai/lib/python3.12/

20:57:00 - LiteLLM:ERROR: vertex_llm_base.py:550 - Failed to load vertex credentials. Check to see if credentials containing partial/invalid information. Error: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.
Traceback (most recent call last):
  File "/Users/haoquanzhai/miniforge3/envs/apple-ai/lib/python3.12/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", line 546, in get_access_token
    _credentials, credential_project_id = self.load_auth(
                                          ^^^^^^^^^^^^^^^
  File "/Users/haoquanzhai/miniforge3/envs/apple-ai/lib/python3.12/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", line 121, in load_auth
    creds, creds_project_id = self._credentials_from_default_auth(
                              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/haoquanzhai/miniforge3/envs/apple-ai/lib/python3.12/site-pack